The taxi trip duration prediction can help people to plan their trips ahead of time.

Problem - Which variables will help in predicting the trip duration correctly?

The trip duration is mainly a function of road network, speed, weather, time of the day, season, weekday or weekend, month.
I have not used the road network data for the model. However, the other variables and their data is available. The taxi dataset, along with the weather data is used to create the predictive model

In [242]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scipy.stats as sc
import datetime as dt
import shapely as sh
import geopandas as gp
import pysal as ps
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.spatial.distance import cdist, pdist
from sklearn.model_selection import GridSearchCV as GSC

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


plt.rcParams['figure.figsize'] = [16, 10]
%matplotlib inline

In [216]:
train = pd.read_csv("./Cleaned_data/train.csv")
train.drop(['Unnamed: 0'], axis=1, inplace=True)

test = pd.read_csv("./Cleaned_data/test.csv")
test.drop(['Unnamed: 0'], axis=1, inplace=True)

In [217]:
nyct = gp.read_file('./nyct2010_17c/nyct2010.shp')
nyct.to_crs(epsg=4326, inplace=True)
nyct = nyct[['BoroCT2010', 'BoroName', 'geometry']]
manh_ct = nyct[nyct.BoroName == 'Manhattan']
manh_ct.BoroCT2010 = manh_ct.BoroCT2010.astype('int')

In [218]:
train_p = pd.read_csv("./Cleaned_data/tp.csv")
train_d = pd.read_csv("./Cleaned_data/td.csv")
test_p = pd.read_csv("./Cleaned_data/testp.csv")
test_d = (pd.read_csv("./Cleaned_data/testd.csv"))

train_p.drop(['Unnamed: 0'], axis=1, inplace=True)
train_d.drop(['Unnamed: 0'], axis=1, inplace=True)
test_p.drop(['Unnamed: 0'], axis=1, inplace=True)
test_d.drop(['Unnamed: 0'], axis=1, inplace=True)

train_p = pd.merge(train_p, manh_ct, how='inner', on='BoroCT2010')
train_d = pd.merge(train_d, manh_ct, how='inner', on='BoroCT2010')
test_p = pd.merge(test_p, manh_ct, how='inner', on='BoroCT2010')
test_d = pd.merge(test_d, manh_ct, how='inner', on='BoroCT2010')

train_p = train_p[['id', 'BoroCT2010']]
train_p.columns = ['id', 'pickup_ct']

train_d = train_d[['id', 'BoroCT2010']]
train_d.columns = ['id', 'dropoff_ct']

test_p = test_p[['id', 'BoroCT2010']]
test_p.columns = ['id', 'pickup_ct']

test_d = test_d[['id', 'BoroCT2010']]
test_d.columns = ['id', 'dropoff_ct']

train_ct = pd.merge(train_p, train_d, how='inner', on='id')
test_ct = pd.merge(test_p, test_d, how='inner', on='id')

test = pd.merge(test, test_ct, how='inner', on='id')
train = pd.merge(train, train_ct, how='inner', on='id')

# Extract the temporal variables

In [219]:
train['pickup_datetime'] = pd.to_datetime(train.pickup_datetime)
train['dropoff_datetime'] = pd.to_datetime(train.dropoff_datetime)

test['pickup_datetime'] = pd.to_datetime(test.pickup_datetime)

train['pickup_date'] = train.pickup_datetime.dt.date
test['pickup_date'] = test.pickup_datetime.dt.date

train['hourofday'] = (train.pickup_datetime.dt.hour*60.0 + train.pickup_datetime.dt.minute)/60.0
train['Dayofweek'] = train.pickup_datetime.dt.dayofweek
train['monthofyear'] = train.pickup_datetime.dt.month
train['dayofmonth'] = train.pickup_datetime.dt.day

test['hourofday'] = test.pickup_datetime.dt.hour
test['Dayofweek'] = test.pickup_datetime.dt.dayofweek
test['monthofyear'] = test.pickup_datetime.dt.month
test['dayofmonth'] = test.pickup_datetime.dt.day

# Calculate the haversine distance and direction, eucledian would also have worked  

In [220]:
# Function taken from - https://www.kaggle.com/karelrv/nyct-from-a-to-z-with-xgboost-tutorial/notebook

def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h


def bearing_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [221]:
train.loc[:, 'distance_haversine'] = haversine_array(train['pickup_latitude'].values, 
                                                     train['pickup_longitude'].values, 
                                                     train['dropoff_latitude'].values, 
                                                     train['dropoff_longitude'].values)

test.loc[:, 'distance_haversine'] = haversine_array(test['pickup_latitude'].values, 
                                                    test['pickup_longitude'].values, 
                                                    test['dropoff_latitude'].values, 
                                                    test['dropoff_longitude'].values)    

train.loc[:, 'direction'] = bearing_array(train['pickup_latitude'].values, 
                                          train['pickup_longitude'].values, 
                                          train['dropoff_latitude'].values, 
                                          train['dropoff_longitude'].values)

test.loc[:, 'direction'] = bearing_array(test['pickup_latitude'].values, 
                                         test['pickup_longitude'].values, 
                                         test['dropoff_latitude'].values, 
                                         test['dropoff_longitude'].values)

In [222]:
train.store_and_fwd_flag = (train.store_and_fwd_flag == 'Y').astype('int')
test.store_and_fwd_flag = (test.store_and_fwd_flag == 'Y').astype('int')

The temporal characteristics of the pickups, dropoffs during the weekday and weekend taken from the 'Hourly pattern Spatial clustering' notebook

In [231]:
temporal = pd.read_csv('./Cleaned_data/temporal.csv')

In [233]:
temporal.columns

Index([u'Unnamed: 0', u'BoroCT2010', u'morning_pickups_wd_trip_dur',
       u'morning_pickups_wd', u'afternoon_pickups_wd_trip_dur',
       u'afternoon_pickups_wd', u'evening_pickups_wd_trip_dur',
       u'evening_pickups_wd', u'night_pickups_wd_trip_dur',
       u'night_pickups_wd', u'morning_dropoffs_wd_trip_dur',
       u'morning_dropoffs_wd', u'afternoon_dropoffs_wd_trip_dur',
       u'afternoon_dropoffs_wd', u'evening_dropoffs_wd_trip_dur',
       u'evening_dropoffs_wd', u'night_dropoffs_wd_trip_dur',
       u'night_dropoffs_wd', u'morning_pickups_we_trip_dur',
       u'morning_pickups_we', u'afternoon_pickups_we_trip_dur',
       u'afternoon_pickups_we', u'evening_pickups_we_trip_dur',
       u'evening_pickups_we', u'night_pickups_we_trip_dur',
       u'night_pickups_we', u'morning_dropoffs_we_trip_dur',
       u'morning_dropoffs_we', u'afternoon_dropoffs_we_trip_dur',
       u'afternoon_dropoffs_we', u'evening_dropoffs_we_trip_dur',
       u'evening_dropoffs_we', u'night_dropoff

In [234]:
temporal = temporal[[u'BoroCT2010', u'morning_pickups_wd',  u'afternoon_pickups_wd', 
                     u'evening_pickups_wd',
       u'night_pickups_wd', 
       u'morning_dropoffs_wd',
       u'afternoon_dropoffs_wd', 
       u'evening_dropoffs_wd', 
       u'night_dropoffs_wd', 
       u'morning_pickups_we', 
       u'afternoon_pickups_we', 
       u'evening_pickups_we', 
       u'night_pickups_we',
       u'morning_dropoffs_we',
       u'afternoon_dropoffs_we', 
       u'evening_dropoffs_we',
       u'night_dropoffs_we']]

In [237]:
train  = pd.merge(train, temporal, left_on='pickup_ct', right_on='BoroCT2010')
train  = pd.merge(train, temporal, left_on='dropoff_ct', right_on='BoroCT2010')

# Weather data

I have taken the average temperature, snowfall and precipitation as predictors. Taxi speed does depend on the precipitation and snowfall as well.

In [227]:
weather = pd.read_csv("./weather_data_nyc_centralpark_2016.csv")

weather.date = pd.to_datetime(weather.date).dt.date

weather = weather[['date', 'average temperature', 'precipitation', 'snow fall']]

weather.loc[weather.precipitation == 'T','precipitation'] = 0.001

weather.loc[weather['snow fall'] == 'T','snow fall'] = 0.001

weather.precipitation = weather.precipitation.astype('float')
weather['snow fall'] = weather['snow fall'].astype('float')

In [230]:
train = pd.merge(train, weather, left_on='pickup_date', right_on='date')

In [240]:
train.columns

Index([u'id', u'vendor_id', u'pickup_datetime', u'dropoff_datetime',
       u'passenger_count', u'pickup_longitude', u'pickup_latitude',
       u'dropoff_longitude', u'dropoff_latitude', u'store_and_fwd_flag',
       u'trip_duration', u'pickup_ct', u'dropoff_ct', u'pickup_date',
       u'hourofday', u'Dayofweek', u'monthofyear', u'dayofmonth',
       u'distance_haversine', u'direction', u'date', u'average temperature',
       u'precipitation', u'snow fall', u'BoroCT2010_x',
       u'morning_pickups_wd_x', u'afternoon_pickups_wd_x',
       u'evening_pickups_wd_x', u'night_pickups_wd_x',
       u'morning_dropoffs_wd_x', u'afternoon_dropoffs_wd_x',
       u'evening_dropoffs_wd_x', u'night_dropoffs_wd_x',
       u'morning_pickups_we_x', u'afternoon_pickups_we_x',
       u'evening_pickups_we_x', u'night_pickups_we_x',
       u'morning_dropoffs_we_x', u'afternoon_dropoffs_we_x',
       u'evening_dropoffs_we_x', u'night_dropoffs_we_x', u'BoroCT2010_y',
       u'morning_pickups_wd_y', u'aftern

In [241]:
train.dropna(inplace=True)

I am training a random forest model, I have not tuned the parameters here, since the computation time is too high.
The model trains a subset of the train data provided and used the remaining train data to test the prediction. The test data provided does not have trip duration, it is not used for testing

In [243]:
X = train[[u'vendor_id',
       u'passenger_count',
       u'store_and_fwd_flag',
       u'hourofday', u'Dayofweek', u'monthofyear', u'dayofmonth',
       u'distance_haversine', u'direction', u'average temperature',
       u'precipitation', u'snow fall',
       u'morning_pickups_wd_x', u'afternoon_pickups_wd_x',
       u'evening_pickups_wd_x', u'night_pickups_wd_x',
       u'morning_dropoffs_wd_x', u'afternoon_dropoffs_wd_x',
       u'evening_dropoffs_wd_x', u'night_dropoffs_wd_x',
       u'morning_pickups_we_x', u'afternoon_pickups_we_x',
       u'evening_pickups_we_x', u'night_pickups_we_x',
       u'morning_dropoffs_we_x', u'afternoon_dropoffs_we_x',
       u'evening_dropoffs_we_x', u'night_dropoffs_we_x',
       u'morning_pickups_wd_y', u'afternoon_pickups_wd_y',
       u'evening_pickups_wd_y', u'night_pickups_wd_y',
       u'morning_dropoffs_wd_y', u'afternoon_dropoffs_wd_y',
       u'evening_dropoffs_wd_y', u'night_dropoffs_wd_y',
       u'morning_pickups_we_y', u'afternoon_pickups_we_y',
       u'evening_pickups_we_y', u'night_pickups_we_y',
       u'morning_dropoffs_we_y', u'afternoon_dropoffs_we_y',
       u'evening_dropoffs_we_y', u'night_dropoffs_we_y']]

y = train[['trip_duration']]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=4)

# param_grid = {'n_estimators' = 100, 'max_features'='sqrt', 'max_leaf_nodes' : range(2,25)}
# gr=GSC(rf1,param_grid=param_grid)
# rf = gr.fit(X_train, Y_train)

regr_rf = RandomForestRegressor(n_estimators=100, random_state=2)
regr_rf.fit(X_train, y_train)

# Predict on new data
y_rf = regr_rf.predict(X_test)

y_rf

array([ 1220.11,   585.33,  1088.79, ...,   325.55,   296.34,   561.16])

In [245]:
def rmsle(predicted,real):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

In [246]:
rmsle(y_rf, y_test.values)

array([ 0.34888875])

The metric used for testing the prediction is "Root mean squared log error". This metric is used, since the data available has some wrong trip durations which are high. The log terms in the metric does not give high weight to large error value, hence it is useful in this case to test our prediction

In [247]:
feat_imp = {}
for idx, feat in enumerate(regr_rf.feature_importances_):
    feat_imp[X_test.columns[idx]] = feat

sorted(feat_imp.items(), key=lambda x: x[1])

[('store_and_fwd_flag', 0.0004335677949985082),
 ('snow fall', 0.0018016095636730792),
 ('vendor_id', 0.0030700146257975195),
 ('afternoon_pickups_we_y', 0.0043009891791907845),
 ('afternoon_pickups_wd_x', 0.0046797959354161608),
 ('evening_dropoffs_we_y', 0.0046805545639881669),
 ('afternoon_dropoffs_wd_x', 0.0047524916321065981),
 ('afternoon_dropoffs_we_y', 0.0047723380498469019),
 ('evening_dropoffs_we_x', 0.0048286807822754442),
 ('afternoon_pickups_we_x', 0.0048496471983468618),
 ('evening_pickups_we_x', 0.0048804088212757299),
 ('afternoon_dropoffs_we_x', 0.0049557608351794508),
 ('evening_dropoffs_wd_y', 0.0050545346995461062),
 ('afternoon_dropoffs_wd_y', 0.0051427189344980476),
 ('morning_dropoffs_we_y', 0.0051802570201373585),
 ('evening_dropoffs_wd_x', 0.00524006067663453),
 ('afternoon_pickups_wd_y', 0.0053190031085163349),
 ('morning_dropoffs_we_x', 0.0055837277899482137),
 ('evening_pickups_we_y', 0.0056676787630421136),
 ('morning_pickups_we_y', 0.0057046767659042399),


The important features we get are distance, direction and the time variables. The snow fall, vendor id, store and fwd flag are not contributing much to the trip duration. The variables created for morning, evening, night, afternoon could have been handled more elegantly. The pickups and dropoffs could have been segregated as morning and evening and weekday, weekend, that could have been more better. The importance of those temporal variables may be relative and can change in different iterations

The above model was created with 100 estimators, i.e 100 random trees were created, that was a computationally expensive and taking more time, so in the below model, I have reduced the number of estimators to 10 and removed the less important features. The rmsle has increased but that can be because of the reduced estimators

In [248]:
X = train[[u'passenger_count',
           u'hourofday', u'Dayofweek', u'monthofyear', u'dayofmonth',
       u'distance_haversine', u'direction', u'average temperature',
       u'precipitation', u'morning_pickups_wd_x', u'afternoon_pickups_wd_x',
       u'evening_pickups_wd_x', u'night_pickups_wd_x',
       u'morning_dropoffs_wd_x', u'afternoon_dropoffs_wd_x',
       u'evening_dropoffs_wd_x', u'night_dropoffs_wd_x',
       u'morning_pickups_we_x', u'afternoon_pickups_we_x',
       u'evening_pickups_we_x', u'night_pickups_we_x',
       u'morning_dropoffs_we_x', u'afternoon_dropoffs_we_x',a
       u'evening_dropoffs_we_x', u'night_dropoffs_we_x',
       u'morning_pickups_wd_y', u'afternoon_pickups_wd_y',
       u'evening_pickups_wd_y', u'night_pickups_wd_y',
       u'morning_dropoffs_wd_y', u'afternoon_dropoffs_wd_y',
       u'evening_dropoffs_wd_y', u'night_dropoffs_wd_y',
       u'morning_pickups_we_y', u'afternoon_pickups_we_y',
       u'evening_pickups_we_y', u'night_pickups_we_y',
       u'morning_dropoffs_we_y', u'afternoon_dropoffs_we_y',
       u'evening_dropoffs_we_y', u'night_dropoffs_we_y']]

y = train[['trip_duration']]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=4)

max_depth = 30

regr_rf = RandomForestRegressor(max_depth=max_depth, random_state=2)
regr_rf.fit(X_train, y_train)

# Predict on new data
y_rf = regr_rf.predict(X_test)

rmsle(y_rf, y_test.values)

array([ 0.36560705])

In [249]:
feat_imp = {}
for idx, feat in enumerate(regr_rf.feature_importances_):
    feat_imp[X_test.columns[idx]] = feat

sorted(feat_imp.items(), key=lambda x: x[1])

[('afternoon_pickups_we_y', 0.0040900255780398674),
 ('evening_dropoffs_we_y', 0.0046903922481110012),
 ('afternoon_pickups_wd_x', 0.0047790221493969696),
 ('afternoon_dropoffs_wd_x', 0.0047871267267194234),
 ('afternoon_dropoffs_wd_y', 0.0048786908114831331),
 ('afternoon_dropoffs_we_y', 0.0048838054589743288),
 ('evening_pickups_we_x', 0.0048846124686470205),
 ('evening_dropoffs_we_x', 0.0049300215369272047),
 ('afternoon_pickups_wd_y', 0.0049334423791533881),
 ('afternoon_dropoffs_we_x', 0.0050129397396574829),
 ('evening_dropoffs_wd_y', 0.0050159933581870823),
 ('afternoon_pickups_we_x', 0.0050534245613028574),
 ('evening_dropoffs_wd_x', 0.0051777120959275889),
 ('morning_dropoffs_we_y', 0.0052931458408999521),
 ('morning_dropoffs_we_x', 0.0057595936307841596),
 ('morning_pickups_we_y', 0.005791092642840604),
 ('night_pickups_wd_y', 0.0058156659446132542),
 ('morning_pickups_wd_y', 0.0058248800319478655),
 ('night_pickups_wd_x', 0.0059623843115627758),
 ('evening_pickups_we_y', 0.0

The trip duration can be predicted from the selected features, more imporvment can be made to the models. But this is more of a network analysis problem and can be solved more accurately by network analysis